In [ ]:
from google.colab import userdata

In [ ]:
openai_apikey = userdata.get("openai_apikey")

In [3]:
%%shell
pip install openai langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.9/221.9 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.2/46.2 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is n

In [5]:
from langchain.llms import OpenAI # modelo generativo
from langchain.chat_models import ChatOpenAI # modelo de chat
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (
    PromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate,
    HumanMessagePromptTemplate, MessagesPlaceholder
)
from langchain.chains import LLMChain, SequentialChain
import pandas as pd

In [ ]:
llm1 = OpenAI(openai_api_key=openai_apikey)

In [ ]:
llm2 = ChatOpenAI(openai_api_key=openai_apikey)

In [ ]:
llm1("Albert Einstein was")

' a German physicist and Nobel Prize winner who is widely considered one of the most influential scientists of the 20th century. He developed the theory of relativity, one of the two pillars of modern physics. His work is also known for its influence on the philosophy of science. Einstein is best known in popular culture for his mass–energy equivalence formula E = mc2. He received the 1921 Nobel Prize in Physics "for his services to theoretical physics, and especially for his discovery of the law of the photoelectric effect".'

In [ ]:
llm2.predict("Hello")

'Hi there! How can I help you today?'

# Modelo Generativo

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history")

In [ ]:
memory

ConversationBufferMemory(memory_key='chat_history')

In [ ]:
memory.chat_memory.add_user_message("Hello, My name is Juan")
memory.chat_memory.add_ai_message("Hello, Mi name is Dyllan. Nice to meet you.")

In [ ]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hello, My name is Juan'), AIMessage(content='Hello, Mi name is Dyllan. Nice to meet you.')]), memory_key='chat_history')

In [ ]:
template = """You are a nice person which is having a conversation with a human.

[Previous conversation]
{chat_history}

[New message]
{message}

[Response]
"""

In [ ]:
prompt = PromptTemplate.from_template(template)

In [ ]:
llm = OpenAI(openai_api_key=openai_apikey)

In [ ]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [ ]:
chain.run({"message": "What was my name?"})

'Your name is Juan.'

In [ ]:
memory

ConversationBufferMemory(chat_memory=ChatMessageHistory(messages=[HumanMessage(content='Hello, My name is Juan'), AIMessage(content='Hello, Mi name is Dyllan. Nice to meet you.'), HumanMessage(content='What was my name?'), AIMessage(content='Your name is Juan.')]), memory_key='chat_history')

# Modelo de Chat

In [ ]:
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [ ]:
llm = ChatOpenAI(openai_api_key=openai_apikey)

In [ ]:
memory.chat_memory.add_user_message("Hello, My name is Juan")
memory.chat_memory.add_ai_message("Hello, Mi name is Dyllan. Nice to meet you.")

In [ ]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate
        .from_template("You are a nice person which is having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{message}")
    ]
)

In [ ]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [ ]:
chain.run({"message": "Can you remind me my name?"})

'Of course, your name is Juan.'

# Zero-Shot Prompting

In [ ]:
template = """
Using the following categories: {categories}, classify the following text: {text}
"""

In [ ]:
prompt = PromptTemplate.from_template(template)

In [ ]:
llm = OpenAI(openai_api_key=openai_apikey)

In [ ]:
chain = prompt | llm

In [ ]:
chain.invoke({"categories": "[possitive, neutral, negative]", "text": "I'm feeling sad today because it's raining"})

'\nNegative'

In [ ]:
chain.invoke({"categories": "[possitive, neutral, negative]", "text": "I'm feeling great today, I'm happy to be alive"})

'\nPositive'

In [ ]:
chain.invoke({"categories": "[joy, sadness, anger, fear, disgust, trust, surprise, anticipation]", "text": "I'm feeling great today, I'm happy to be alive"})

'\nJoy'

# One-shot Prompting

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/f/awesome-chatgpt-prompts/main/prompts.csv")
df

,act,prompt
0,SEO Prompt,"Using WebPilot, create an outline for an artic..."
1,Linux Terminal,I want you to act as a linux terminal. I will ...
2,English Translator and Improver,"I want you to act as an English translator, sp..."
3,`position` Interviewer,I want you to act as an interviewer. I will be...
4,JavaScript Console,I want you to act as a javascript console. I w...
...,...,...
161,note-taking assistant,I want you to act as a note-taking assistant f...
162,`language` Literary Critic,I want you to act as a `language` literary cri...
163,Prompt Enhancer,Act as a Prompt Enhancer AI that takes user-in...
164,Cheap Travel Ticket Advisor,You are a cheap travel ticket advisor speciali...


In [ ]:
template = """
{oneshot}

[Previous commands]
{chat_history}

[New command]
{message}

[Response]
"""

In [ ]:
system_msg = df.query("act == 'Linux Terminal'").prompt.iloc[0]
system_msg = ". ".join(system_msg.split(".")[:-2])
system_msg

'I want you to act as a linux terminal.  I will type commands and you will reply with what the terminal should show.  I want you to only reply with the terminal output inside one unique code block, and nothing else.  do not write explanations.  do not type commands unless I instruct you to do so'

In [ ]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template(system_msg),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{message}")
    ]
)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
llm = ChatOpenAI(openai_api_key=openai_apikey)

In [ ]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [ ]:
chain.run({"message": "ls"})

'Desktop  Documents  Downloads  Music  Pictures  Public  Templates  Videos'

In [ ]:
chain.run({"message": "mkdir my_folder"})

''

In [ ]:
chain.run({"message": "ls"})

'Desktop  Documents  Downloads  Music  Pictures  Public  Templates  Videos  my_folder'

In [ ]:
chain.run({"message": "touch my_folder/file.py"})

''

In [ ]:
chain.run({"message": "echo 'print(1)' > my_folder/file.py"})

''

In [ ]:
chain.run({"message": "cat my_folder/file.py"})

'print(1)'

In [ ]:
chain.run({"message": "python my_folder/file.py"})

'1'

In [ ]:
chain.run({"message": "echo 'import numpy as np; x = np.random.normal(size=(10, 10)); print(x)' > my_folder/file.py"})

''

In [ ]:
chain.run({"message": "python my_folder/file.py"})

'[[ 0.0504239   0.07219414 -0.72559325 -0.71600415 -1.27301691 -0.47411045\n  -1.40176272 -0.03738069  0.73910617 -0.67450248]\n [-0.10950544 -0.80207095 -0.12663169  0.43153338  0.25001108 -1.59601617\n  -0.68364667 -0.64604514  1.59797823 -0.33540313]\n [ 0.89843193 -1.14302023  0.55137783  0.98999591 -1.20454135  0.41511403\n   0.00760204 -0.2802799   0.96561698  0.56232632]\n [ 0.09961067 -0.54074161 -0.04297842 -1.6985198   1.29427472 -0.31905039\n   1.83531966  1.54786832 -0.63177484 -0.5170668 ]\n [-0.09833157  0.04220793  0.84052056 -1.49679531 -0.2186147   0.56749854\n   1.58564317  0.49181117 -1.500259   -0.64612887]\n [ 0.42638309 -0.68245127  0.24674001 -0.66267879 -1.03947359  0.79162562\n   1.0853964  -0.23967475  0.1691889   0.37982557]\n [-0.32547096 -0.07290547 -0.52632088 -0.33564822  0.58814848 -1.43113661\n  -0.37880168 -0.38624095  0.19161367 -1.01944211]\n [-1.27413265 -0.36304518 -0.04260122  0.15388646 -1.66334901 -0.16481053\n   0.32101474  0.29697741 -0.313200

# Few-shot Prompting

In [ ]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template("You're a language detector"),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{message}")
    ]
)

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
# two-shot prompt
memory.chat_memory.add_user_message("this is a sample text, and I want to be happy.")
memory.chat_memory.add_ai_message("english")
memory.chat_memory.add_user_message("Petro celebró decisión de juez que impide urbanizar la reserva Thomas Van der Hammen")
memory.chat_memory.add_ai_message("spanish")

In [ ]:
llm = ChatOpenAI(openai_api_key=openai_apikey)

In [ ]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [ ]:
chain.run({"message": "Me sento bien el dia de hoy"})

'spanish'

In [ ]:
chain.run({"message": "I'm doing my homework about machine learning"})

'english'

In [ ]:
chain.run({"message": "Du siehst den Wald vor lauter Bäumen nicht. "})

'german'

# Chain of Thoughts

In [ ]:
llm = ChatOpenAI(openai_api_key=openai_apikey)

In [ ]:
llm.predict(
    "Ayer, Pepito estuvo en reunion con Antonio por 1h, luego trabajo 3h en el proyecto." +
    "Al regresar de su hora de almuerzo, tuvo una reunion hora y media con el CEO. " +
    "Cuantas horas paso Pepito en reuniones?"
    )

'Pepito pasó 1 hora en reunión con Antonio, 3 horas trabajando en el proyecto y 1.5 horas en reunión con el CEO. \n\nEn total, Pepito pasó 1 + 3 + 1.5 = 5.5 horas en reuniones.'

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
memory.chat_memory.add_user_message(
    "Hoy, Maria estuvo implementando el frontend por 4h seguidas, tuvo 1h de almuerzo, " +
    "y al final del dia se reunio con el manager por 2h."
)
memory.chat_memory.add_ai_message(
    "Veamos el paso a paso: extraemos las horas que no son reunion (4h + 5h), " +
    "extraemos las horas que son reunion (2h), por lo tanto, la respuesta es 2h."
)
memory.chat_memory.add_user_message(
    "Ayer me reuni con mis companeros por 4h en la manana, en la tarde me iba a reunir pero mejor trabaje por 3h"
)
memory.chat_memory.add_ai_message(
    "Veamos el paso a paso: extraemos las horas que no son reunion (3h), " +
    "extraemos las horas que son reunion (4h), por lo tanto, la respuesta es 4h."
)

In [ ]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template("Ayudas a calcular horas de reunion."),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{message}")
    ]
)

In [ ]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [ ]:
chain.run({
    "message": "Ayer, Pepito estuvo en reunion con Antonio por 1h, luego trabajo 3h en el proyecto." +
    "Al regresar de su hora de almuerzo, tuvo una reunion hora y media con el CEO. " +
    "Cuantas horas paso Pepito en reuniones?"
    })

'Pepito estuvo en reunión con Antonio por 1 hora, luego tuvo una reunión con el CEO por 1.5 horas. En total, Pepito pasó 2.5 horas en reuniones.'

# Tree of Thoughts

In [ ]:
llm = OpenAI(openai_api_key=openai_apikey)

In [ ]:
template = """
Paso 1: estoy armando una sugerencia de proyecto para un estudiante de computacion, a el le gusta {topic}. Basado en estos temas, dame 5 posibles ejemplos de lo quede desarrollar.
"""
prompt1 = PromptTemplate(
    input_variables=["topic"],
    template=template
)
chain1 = LLMChain(
    llm=llm,
    prompt=prompt,
    output_key="applications"
)

In [ ]:
template = """
Paso 2: para cada una de estas apliaciones: {applications}. Evalua que tanto tiempo podria tomar su desarrollo y que tan novedosas puedan llegar a ser.
"""
prompt2 = PromptTemplate(
    input_variables=["applications"],
    template=template
)
chain2 = LLMChain(
    llm=llm,
    prompt=prompt2,
    output_key="ranking"
)

In [ ]:
template = """
Paso 3: dadas estas aplicaciones: {applications} y estas consideraciones: {ranking}, quiero UNICAMENTE la mejor opcion y para esta UNICA opcion dame un listado 5 de herramientas de software mas importantes para construirla.
"""
prompt3 = PromptTemplate(
    input_variables=["applications", "ranking"],
    template=template
)
chain3 = LLMChain(
    llm=llm,
    prompt=prompt3,
    output_key="tools"
)

In [ ]:
chain = SequentialChain(
    chains=[chain1, chain2, chain3],
    input_variables=["topic"],
    output_variables=["tools"]
)

In [ ]:
print(chain.run({"topic": "computacion evolutiva"}))


1. Scikit-learn
2. TensorFlow
3. Keras
4. PyTorch
5. NLTK


# Self-Consistency

In [64]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_user_message(
    "Es 8 un numero primo?"
)
memory.chat_memory.add_ai_message(
    "Veamos el paso a paso: 8 es divisible por 2, como 2 es distinto de 1 u 8, entonces 8 no es un numero primo."
)
memory.chat_memory.add_user_message(
    "Es 13 un numero primo?"
)
memory.chat_memory.add_ai_message(
    "Veamos el paso a paso: 13 es divisible por 1 y por 13, entonces 13 es un numero primo."
)

In [55]:
prompt = ChatPromptTemplate.from_messages(messages=[
    SystemMessagePromptTemplate.from_template("Eres un modelo que identifica numeros primos."),
    MessagesPlaceholder(variable_name="chat_history"),
    HumanMessagePromptTemplate.from_template("Es {number} un numero primo?")
])

In [56]:
llm = ChatOpenAI(openai_api_key=openai_apikey, temperature=1.1)

In [57]:
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [58]:
chain.run({"number": "7057"})

'Para determinar si 7057 es un número primo, debemos verificar si es divisible por algún número distinto de 1 y de sí mismo. Calculamos la raíz cuadrada de 7057, que está aproximadamente entre 83 y 84. A continuación, realizamos una prueba de división por todos los números primos menores o iguales a 83. Si encontramos algún divisor, sabremos que 7057 no es primo.\n\nRealizando la prueba de división, encontramos que 7057 es divisible por 83, ya que 7057 dividido entre 83 da 85 como resultado sin decimales. Por lo tanto, podemos concluir que 7057 no es un número primo.'

In [60]:
sc_prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate.from_template(
            "Ayudas a dar una conclusion usando un voto de mayoria a partir de varios argumentos"
        ),
        HumanMessagePromptTemplate.from_template("Estas son las oraciones:\n{responses}")
    ]
)

In [61]:
sc_chain = LLMChain(
    llm=llm,
    prompt=sc_prompt
)

In [65]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
memory.chat_memory.add_user_message(
    "Es 8 un numero primo?"
)
memory.chat_memory.add_ai_message(
    "Veamos el paso a paso: 8 es divisible por 2, como 2 es distinto de 1 u 8, entonces 8 no es un numero primo."
)
memory.chat_memory.add_user_message(
    "Es 13 un numero primo?"
)
memory.chat_memory.add_ai_message(
    "Veamos el paso a paso: 13 es divisible por 1 y por 13, entonces 13 es un numero primo."
)
chain = LLMChain(
    llm=llm,
    prompt=prompt,
    memory=memory
)

In [66]:
responses = []
for _ in range(5):
    responses.append(chain.run({"number": "7057"}))
    memory.chat_memory.messages.pop(-1)
    memory.chat_memory.messages.pop(-1)
print(responses)

['Para verificar si 7057 es un número primo, debemos comprobar si es divisible por algún número más pequeño que él. \n\nConsiderando los números primos menores a la raíz cuadrada de 7057, podemos realizar la división y ver si el residuo es igual a 0.\n\nDividiendo 7057 entre los números primos menores a su raíz cuadrada (es decir, hasta aproximadamente 84), encontramos que no es divisible exactamente por ninguno de ellos.\n\nPor lo tanto, en base a esta prueba, podemos concluir que 7057 sí es un número primo.', 'Para verificar si 7057 es un número primo, debemos verificar si es divisible entre algún número natural menor que él mismo. Realizando la verificación, encontramos que 7057 es divisible por 7 y por 1008. Por lo tanto, 7057 no es un número primo.', 'Para determinar si 7057 es un número primo, se puede verificar si es divisible por algún número aparte de 1 y él mismo.\n\nVerifiquemos la divisibilidad de 7057. Comenzamos dividiendo por los primeros números primos (2, 3, 5, 7) y ve

In [73]:
response = "\n-------\n".join(responses)

In [74]:
print(response)

Para verificar si 7057 es un número primo, debemos comprobar si es divisible por algún número más pequeño que él. 

Considerando los números primos menores a la raíz cuadrada de 7057, podemos realizar la división y ver si el residuo es igual a 0.

Dividiendo 7057 entre los números primos menores a su raíz cuadrada (es decir, hasta aproximadamente 84), encontramos que no es divisible exactamente por ninguno de ellos.

Por lo tanto, en base a esta prueba, podemos concluir que 7057 sí es un número primo.
-------
Para verificar si 7057 es un número primo, debemos verificar si es divisible entre algún número natural menor que él mismo. Realizando la verificación, encontramos que 7057 es divisible por 7 y por 1008. Por lo tanto, 7057 no es un número primo.
-------
Para determinar si 7057 es un número primo, se puede verificar si es divisible por algún número aparte de 1 y él mismo.

Verifiquemos la divisibilidad de 7057. Comenzamos dividiendo por los primeros números primos (2, 3, 5, 7) y ve

In [75]:
sc_chain.run({"responses": response})

'Después de evaluar todos los argumentos presentados, se puede llegar a una conclusión utilizando un voto de mayoría. La mayoría de los argumentos respaldan la afirmación de que 7057 es un número primo. Aunque algunos argumentos sostienen que 7057 es divisible por ciertos números, la mayoría de los argumentos, incluyendo la prueba de divisibilidad por números primos y la falta de divisores enteros aparte de 1 y el mismo número, respaldan que 7057 es un número primo. Por lo tanto, concluimos que 7057 es un número primo.'